In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.preprocessing import image_dataset_from_directory
import pathlib

In [3]:
# --------------------
# 1. データセットの読み込み
# --------------------
DATA_DIR = pathlib.Path("dog_cat_photos")
train_dir = DATA_DIR / "train"
test_dir  = DATA_DIR / "test"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 123

train_dataset = image_dataset_from_directory(
    train_dir,
    labels="inferred",
    label_mode="int",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED
)

test_dataset = image_dataset_from_directory(
    test_dir,
    labels="inferred",
    label_mode="int",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

class_names = train_dataset.class_names
print("Classes:", class_names)

Found 300 files belonging to 2 classes.
Found 100 files belonging to 2 classes.
Classes: ['cat', 'dog']


In [4]:
# --------------------
# 2. データ拡張（水増し）
# --------------------
def flip_left_right(image, label):
    return tf.image.flip_left_right(image), label

def flip_up_down(image, label):
    return tf.image.flip_up_down(image), label

def rot90(image, label):
    return tf.image.rot90(image), label

def rot180(image, label):
    return tf.image.rot90(image, k=2), label

def rot270(image, label):
    return tf.image.rot90(image, k=3), label

# 水増しデータセット作成
train_dataset_lr     = train_dataset.map(flip_left_right)
train_dataset_ud     = train_dataset.map(flip_up_down)
train_dataset_rot90  = train_dataset.map(rot90)
train_dataset_rot180 = train_dataset.map(rot180)
train_dataset_rot270 = train_dataset.map(rot270)

# 結合
train_dataset = (train_dataset
    .concatenate(train_dataset_lr)
    .concatenate(train_dataset_ud)
    .concatenate(train_dataset_rot90)
    .concatenate(train_dataset_rot180)
    .concatenate(train_dataset_rot270)
)

In [5]:
# シャッフル + パイプライン最適化
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.shuffle(1000).prefetch(AUTOTUNE)
test_dataset  = test_dataset.prefetch(AUTOTUNE)


In [6]:
# --------------------
# 3. モデル構築
# --------------------
input_layer = tf.keras.Input(shape=(224, 224, 3))
l_layer = mobilenet_v2.preprocess_input(input_layer)

base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    input_tensor=l_layer,
    include_top=False,
    weights="imagenet",
    pooling="avg"
)
base_model.trainable = False  # 転移学習: ベースは凍結

output_layer = tf.keras.layers.Dense(1, activation="sigmoid")

model = tf.keras.Sequential([
    base_model,
    output_layer
])

model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 1280)           │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [7]:
# --------------------
# 4. 学習
# --------------------
history = model.fit(
    train_dataset,
    epochs=20,
    validation_data=test_dataset
)


Epoch 1/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 8s 109ms/step - accuracy: 0.8089 - loss: 0.4263 - val_accuracy: 0.9700 - val_loss: 0.1299
Epoch 2/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 105ms/step - accuracy: 0.9156 - loss: 0.2225 - val_accuracy: 0.9600 - val_loss: 0.0800
Epoch 3/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 106ms/step - accuracy: 0.9444 - loss: 0.1692 - val_accuracy: 0.9700 - val_loss: 0.0754
Epoch 4/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 108ms/step - accuracy: 0.9494 - loss: 0.1445 - val_accuracy: 0.9700 - val_loss: 0.0759
Epoch 5/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 108ms/step - accuracy: 0.9661 - loss: 0.1182 - val_accuracy: 0.9600 - val_loss: 0.0744
Epoch 6/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 111ms/step - accuracy: 0.9739 - loss: 0.1026 - val_accuracy: 0.9600 - val_loss: 0.0688
Epoch 7/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 110ms/step - accuracy: 0.9733 - loss: 0.0892 - val_accuracy: 0.9600 - val_loss: 0.0705
Epoch 8/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 112ms/step - accuracy: 0.9772 - loss: 0.0808 - val_accuracy: 0.

In [8]:
# --------------------
# 5. 評価
# --------------------
results = model.evaluate(test_dataset, return_dict=True)
print("Evaluation:", results)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.9600 - loss: 0.0691 
Evaluation: {'accuracy': 0.9599999785423279, 'loss': 0.06906052678823471}
